In [1]:
# reference: http://cs229.stanford.edu/notes/cs229-notes1.pdf
import numpy as np
#import matplotlib as ml
#import matplotlib.pyplot as plt
import random
import sys

data = np.loadtxt('assign1_data.txt')
#print data

x1 = data[:, 0]
print ("x1:", x1)

x2 = data[:, 1]
print ("x2:", x2)

z = data[:, 3]
print ("z:", z)

'''
solution from part1 is:
w1:	-2.04424259514
w2:	3.99686016866
b:	-0.924290811868
'''

('x1:', array([ 0.1227,  0.3914,  0.7725,  0.8342,  0.5084,  0.9983,  0.6643,
        0.0954,  0.6949,  0.6075,  0.3717,  0.0193,  0.9314,  0.2206,
        0.0903,  0.5466,  0.4331,  0.5293,  0.8955,  0.0701,  0.0771,
        0.1972,  0.6405,  0.3203,  0.103 ,  0.5605,  0.4288,  0.7101,
        0.6575,  0.9498,  0.555 ,  0.0947,  0.4293,  0.31  ,  0.7297,
        0.1973,  0.1571,  0.2272,  0.2099,  0.3972,  0.0033,  0.6441,
        0.5549,  0.9193,  0.5341,  0.1555,  0.1839,  0.9949,  0.7918,
        0.4088,  0.2247,  0.3901,  0.4309,  0.0732,  0.8077,  0.0854,
        0.8955,  0.8794,  0.1094,  0.0726,  0.4377,  0.889 ,  0.5791,
        0.9721,  0.5719,  0.3156,  0.8065,  0.4777,  0.6264,  0.2117,
        0.042 ,  0.0916,  0.6035,  0.6354,  0.4102,  0.695 ,  0.2153,
        0.0768,  0.2356,  0.1591,  0.7171,  0.9988,  0.3936,  0.0201,
        0.6295,  0.1771,  0.5888,  0.5302,  0.3434,  0.7154,  0.3506,
        0.5432,  0.4687,  0.013 ,  0.1311,  0.697 ,  0.3996,  0.2208,
        0.11

'\nsolution from part1 is:\nw1:\t-2.04424259514\nw2:\t3.99686016866\nb:\t-0.924290811868\n'

In [2]:
def plot_error_epoch(accuracy_arr, lr, batch_size):
    accuracy_arr_y = np.array(accuracy_arr)
    epochs = []
    for i in range(len(accuracy_arr)):
        epochs.append(i)
    epochs_x = np.array(epochs)
    plt.plot(epochs_x, accuracy_arr_y)
    title = ''
    title += 'lr:'
    title += str(lr)
    title += ' batch_size:'
    title += str(batch_size)
    plt.title(title)
    plt.xlabel('epoch')
    plt.ylabel('accurary')
    plt.show()

In [3]:
def activation(x): # use sigmoid as activation function
    return 1 / (1 + np.exp(-x))

In [4]:
# create method to train and get result with different settings like online, minibatch, batch and learning rate
def train_and_get_result(x1, x2, lr, batch_size):
    
    if batch_size > 75:
        batch_size = 75
        
    accuracy_arr = []
    
    w1 = random.random() * 3
    w2 = random.random() * 3
    b = random.random() * 3
    
    print "random initial w1:\t", w1
    print "random initial w2:\t", w2
    print "random initial b:\t", b
    
    epoch = 0
    # training set size : validation set size = 3 : 1
    training_set_index = 0
    size = len(x1)
    validation_set_index = 0
    validation_set_index_step = size / 4
    pre_validation_error = sys.maxint
    curr_validation_error = sys.maxint - 1
    while curr_validation_error >= 1 and (pre_validation_error >= curr_validation_error or curr_validation_error <= 0): # early stopping when the validation error is greater than its previous one
    #while True:

        validation_set_index += validation_set_index_step
        validation_set_index %= size

        x1_validation = x1[validation_set_index : validation_set_index + validation_set_index_step]
        x2_validation = x2[validation_set_index : validation_set_index + validation_set_index_step]
        z_validation = z[validation_set_index : validation_set_index + validation_set_index_step]

        # training
        i = 1
        for x1i, x2i, zi in zip(x1, x2, z):
            sum_error_w1 = 0
            sum_error_w2 = 0
            sum_error_b = 0
            if x1i not in zip(x1_validation) and x2i not in zip(x2_validation) and zi not in zip(z_validation):
                error = zi - activation(x1i * w1 + x2i * w2 + b)
                sum_error_w1 += error * x1i
                sum_error_w2 += error * x2i
                sum_error_b += error
                if i >= batch_size: # only update weights after one batch
                    w1 = w1 + lr * sum_error_w1
                    w2 = w2 + lr * sum_error_w2
                    b = b + lr * sum_error_b
                    sum_error_w1 = 0
                    sum_error_w2 = 0
                    sum_error_b = 0
                    i = 1
                    continue
                i += 1

        if epoch >= 4 and epoch % 4 == 0:
            pre_validation_error = curr_validation_error
            if epoch % 100 == 0: # print only every 100 epochs
                print "epoch:\t", epoch, "\t curr_validation_error:\t", curr_validation_error
            curr_validation_error = 0

        # validation
        for x1i, x2i, zi in zip(x1_validation, x2_validation, z_validation):
            curr_validation_error += abs(zi - (x1i * w1 + x2i * w2 + b))
            
        # get error from the entire set for later plotting
        accuracy = 0
        for x1i, x2i, yi in zip(x1, x2, z):
            if zi == activation(x1i * w1 + x2i * w2 + b):
                accurary += 1
        accuracy /= len(x1)
        accuracy_arr.append(accurary)
        epoch += 1

    plot_error_epoch(accurary, lr, batch_size)
    print "Result: pre_validation_error:\t", pre_validation_error, "curr_validation_error:\t", curr_validation_error
    print "Result: w1:\t", w1, "\tw2:\t", w2, "\tb:\t", b, "\tnum of epoch:\t", epoch

In [ ]:
# online
# learning rate lr = 0.001
train_and_get_result(x1, x2, lr = 0.001, batch_size = 1)